In [ ]:
from IPython.display import display, Markdown

notebook_4_code = """
# 🔥 NOTEBOOK 4 - API Simulation & WhatsApp Communication (PREMIUM VERSION)

# ✅ Step 1: Install required libraries
!pip install flask-ngrok twilio --quiet

# ✅ Step 2: Import necessary modules
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
from flask_ngrok import run_with_ngrok
import json, os
from datetime import datetime

# ✅ Step 3: Initialize app
app = Flask(__name__)
run_with_ngrok(app)

# ✅ Step 4: Load hospital database (simulate a rich one)
hospitales_por_region = {
    "mexico": [
        "Médica Sur", "Hospital Ángeles del Pedregal", "Hospital ABC",
        "Centro Médico Dalinde", "Star Médica"
    ],
    "estados unidos": [
        "Mayo Clinic", "Cleveland Clinic", "Johns Hopkins Hospital"
    ],
    "francia": [
        "Hôpital Pitié-Salpêtrière", "Hôpital Necker", "Clinique Ambroise Paré"
    ],
    "japon": [
        "Tokyo Medical University Hospital", "Keio University Hospital"
    ],
    "brasil": [
        "Hospital Albert Einstein", "Hospital Sírio-Libanês"
    ],
    "sudafrica": [
        "Netcare Milpark Hospital", "Life Fourways Hospital"
    ]
}

# ✅ Step 5: Pacientes con contexto de estado
PACIENTES_FILE = "pacientes.json"
def cargar_pacientes():
    if os.path.exists(PACIENTES_FILE):
        with open(PACIENTES_FILE, "r") as f:
            return json.load(f)
    return {}

def guardar_pacientes(data):
    with open(PACIENTES_FILE, "w") as f:
        json.dump(data, f, indent=2)

pacientes = cargar_pacientes()

# ✅ Step 6: Endpoint principal
@app.route("/whatsapp", methods=["POST"])
def whatsapp():
    numero = request.form.get("From")
    mensaje = request.form.get("Body").strip().lower()
    respuesta = MessagingResponse()
    twilio_msg = respuesta.message()

    if numero not in pacientes:
        pacientes[numero] = {
            "estado": "inicio", "nombre": None, "region": None,
            "hospital": None, "cita": None, "idioma": "es"
        }
        guardar_pacientes(pacientes)

    user = pacientes[numero]

    if mensaje in ["hi", "hello"]:
        user["idioma"] = "en"
        user["estado"] = "menu"
        twilio_msg.body("👋 Hi! I’m your premium medical assistant.\nWhat do you need?\n\n1. Emergency 🚨\n2. Book an appointment 📅\n3. See hospital options 🏥")
        guardar_pacientes(pacientes)
        return str(respuesta)

    if "hola" in mensaje:
        user["idioma"] = "es"
        user["estado"] = "menu"
        twilio_msg.body("👋 ¡Hola! Soy tu *asistente médico premium*.\n¿Qué necesitas?\n\n1. Tengo una emergencia 🚨\n2. Agendar cita 📅\n3. Ver hospitales 🏥")
        guardar_pacientes(pacientes)
        return str(respuesta)

    idioma = user["idioma"]

    if mensaje == "1":
        user["estado"] = "esperando_region"
        msg = "🌍 ¿En qué región estás? (Ej. México, Francia, Japón, Sudáfrica)" if idioma == "es" else "🌍 What region are you in? (e.g. Mexico, France, Japan, South Africa)"
        twilio_msg.body(msg)
        guardar_pacientes(pacientes)

    elif user["estado"] == "esperando_region":
        if mensaje in hospitales_por_region:
            user["region"] = mensaje
            user["estado"] = "esperando_hospital"
            lista = hospitales_por_region[mensaje]
            opciones = "\n".join([f"{i+1}. {h}" for i, h in enumerate(lista)])
            msg = f"🏥 Hospitales disponibles en {mensaje.title()}:\n{opciones}\n\nResponde con el número." if idioma == "es" else f"🏥 Hospitals in {mensaje.title()}:\n{opciones}\n\nReply with the number."
            twilio_msg.body(msg)
        else:
            msg = "Región no reconocida. Intenta con otra." if idioma == "es" else "Region not recognized. Try again."
            twilio_msg.body(msg)
        guardar_pacientes(pacientes)

    elif user["estado"] == "esperando_hospital" and mensaje.isdigit():
        idx = int(mensaje) - 1
        region = user["region"]
        lista = hospitales_por_region.get(region, [])
        if 0 <= idx < len(lista):
            user["hospital"] = lista[idx]
            user["estado"] = "menu"
            msg = f"✅ Te canalizamos a *{lista[idx]}*. El equipo fue notificado." if idioma == "es" else f"✅ You've been routed to *{lista[idx]}*. Medical team notified."
            twilio_msg.body(msg)
        else:
            msg = "Número inválido. Intenta de nuevo." if idioma == "es" else "Invalid number. Try again."
            twilio_msg.body(msg)
        guardar_pacientes(pacientes)

    else:
        fallback = "No entendí tu mensaje. Escribe 'hola' o 'hi'." if idioma == "es" else "I didn’t understand that. Please say 'hi' or 'hello'."
        twilio_msg.body(fallback)

    return str(respuesta)

# ✅ Step 7: Run the app
app.run()
"""

display(Markdown("### ✅ Aquí tienes el código completo del Notebook 4 listo para copiar en Google Colab"))
print(notebook_4_code)
